You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__ are currently saved in `office_addresses.csv`. If the value for office is `NaN`, then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}

### Unzipping File using zipfile

In [1]:
import zipfile
try:
    with zipfile.ZipFile ("D:\\Datacamp\\Consolidating_Employee_Data\\datasets.zip", 'r') as zObj:
        zObj.extractall("D:\\Datacamp\\Consolidating_Employee_Data\\")
except:
    print("Already Unzipped")


In [2]:
import pandas as pd

### Reading office_addresses.csv

In [3]:
offices = pd.read_csv("datasets/office_addresses.csv")
df_offices_columns = offices.columns
print("Offices Address")
offices


Offices Address


,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


### Reading both sheets of employee_information.xlsx

### 1. Reading data from sheet 1 of employee information 

In [4]:
employee_information = pd.read_excel("datasets/employee_information.xlsx")#, sheet_name="emergency_contacts",)
print("Employee Information")
employee_information

Employee Information


,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


### 2. Reading data from sheet 2 of employee information, i.e., emergency contact

In [5]:
emergency_col_names = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number','relationship']
# Usually header = None is not required because names tag will superseed it but I have used it because without header top row was being replaced by names.
emergency_contacts = pd.read_excel("datasets/employee_information.xlsx", sheet_name = "emergency_contacts", header = None, names = emergency_col_names)
print("\nEmergency Contact Information")
emergency_contacts


Emergency Contact Information


,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


### Reading data from employee_roles.json

In [6]:
employee_roles = pd.read_json("datasets\employee_roles.json", orient= 'index')
employee_roles = employee_roles.reset_index().rename(columns={"index":'employee_id'})
print("Employee Roles")
employee_roles

Employee Roles


,employee_id,title,monthly_salary,team
0,A2R5H9,CEO,$4500,Leadership
1,H8K0L6,CFO,$4500,Leadership
2,G4R7V0,Business Developer,$3000,Sales
3,M1Z7U9,Office Manager,$2000,People Operations


### Assigning employee_id column as the index column in 3 Dataframes to use Join function
Join function is used to join several dataframes w.r.t. keys/indexs 

In [7]:
employee_information = employee_information.set_index('employee_id')
emergency_contacts = emergency_contacts.set_index('employee_id')
employee_roles = employee_roles.set_index('employee_id')


### Joining above 3 Dataframes using Join function and Popping out duplicate name columns

In [8]:
info_eme = emergency_contacts.join(employee_information, how='right')
info_eme_role = info_eme.join(employee_roles, how = 'left')
info_eme_role.pop('employee_last_name')
info_eme_role.pop('employee_first_name')
info_eme_role

,last_name,first_name,emergency_contact,emergency_contact_number,relationship,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team
employee_id,,,,,,,,,,,,
A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership
H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,GB,London,Baker Street,221,CFO,$4500,Leadership
G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,US,New-York,Perry Street,66,Business Developer,$3000,Sales
M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations


### Resetting Index to prevent Vanishing of index after Merge function

In [9]:
info_eme_role = info_eme_role.reset_index()
info_eme_role_ofc = info_eme_role.merge(offices, how ='left', left_on = 'employee_country', right_on = 'office_country')
info_eme_role_ofc.set_index('employee_id')

,last_name,first_name,emergency_contact,emergency_contact_number,relationship,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,GB,London,Baker Street,221,CFO,$4500,Leadership,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,US,New-York,Perry Street,66,Business Developer,$3000,Sales,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,NaN,NaN,NaN,NaN,NaN


### Assingning Remote to NaN values in Office Dataframe columns 

In [10]:
for col in ["office", "office_country", "office_city", "office_street", "office_street_number"]:
    info_eme_role_ofc[col].fillna("Remote", inplace=True)
info_eme_role_ofc

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,GB,London,Baker Street,221,CFO,$4500,Leadership,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,US,New-York,Perry Street,66,Business Developer,$3000,Sales,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Remote,Remote,Remote,Remote,Remote


### Setting Index to employee_id

In [11]:
info_eme_role_ofc.set_index(info_eme_role_ofc.columns[0])

,last_name,first_name,emergency_contact,emergency_contact_number,relationship,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,GB,London,Baker Street,221,CFO,$4500,Leadership,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,US,New-York,Perry Street,66,Business Developer,$3000,Sales,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Remote,Remote,Remote,Remote,Remote
